## 621. Task Scheduler

### 📝 Description
Given a list of CPU tasks represented by capital letters `A` to `Z`, and a non-negative integer `n` representing the **cooldown** period between two same tasks, return the **least number of time units** the CPU will take to finish all the given tasks.

The same task **must be separated** by at least `n` units of time. During that cooldown, the CPU can either run other tasks or stay idle.

---

### ⚙️ Approach
- Use a **max heap** to always choose the task with the highest remaining frequency.
- Use a **queue** to track tasks that are currently in the cooldown period. Each item in the queue is a tuple `(ready_time, -count)`.
- At each time unit:
  - Increment the current time.
  - If a task in the cooldown queue becomes available (`ready_time == current time`), reinsert it into the heap.
  - If the heap is not empty, run the task with the highest remaining frequency.
  - If the task still has remaining runs, push it into cooldown with the correct future `ready_time`.
- Continue until both the heap and the cooldown queue are empty.

---

### 🧠 Key Concepts
- **Max Heap Simulation**:
  - Python’s `heapq` is a min heap, so we store negative frequencies to simulate a max heap.
- **Cooldown Tracking**:
  - A `deque` is used to track when tasks can be pushed back into the heap.
- **Idle Time**:
  - If no task is ready and the heap is empty, time still moves forward (implicitly accounts for idle periods).
- **Time Complexity**:
  - O(n log n), where n is the number of tasks — due to heap operations and queue handling.
- **Space Complexity**:
  - O(26) for task types + O(n) for cooldown tracking.

---

### 🔍 Example
```python
Input: tasks = ["A","A","A","B","B","B"], n = 2

A -> B -> idle -> A -> B -> idle -> A -> B

Output: 8

In [ ]:
import heapq
from collections import Counter, deque

class Solution:
    def leastInterval(self, tasks: list[str], n: int) -> int:
        # Step 1: Count frequency of each task
        task_counts = Counter(tasks)

        # Step 2: Max heap to always pick the most frequent remaining task
        # Python's heapq is a min-heap, so we store negative counts to simulate a max-heap
        max_heap = [-cnt for cnt in task_counts.values()]
        heapq.heapify(max_heap)

        # Step 3: Queue to store tasks that are in cooldown
        # Each entry: (ready_time, -count)
        cooldown = deque()

        time = 0  # total units of time taken

        while max_heap or cooldown:
            time += 1

            # Step 4: If there's a task ready to come out of cooldown, put it back in heap
            if cooldown and cooldown[0][0] == time:
                _, cnt = cooldown.popleft()
                heapq.heappush(max_heap, cnt)

            # Step 5: If heap is not empty, do the next most frequent task
            if max_heap:
                cnt = heapq.heappop(max_heap)
                cnt += 1  # this task has been run once, so its count goes up (less negative)
                if cnt < 0:
                    # Still has more runs left, go to cooldown
                    cooldown.append((time + n + 1, cnt))
            # else:
                # CPU idles if no task can be done (we already added +1 to time above)

        return time